In [2]:
import pandas as pd
import numpy as np
import os
import datetime
import time
import glob
import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn import metrics 

import warnings
warnings.filterwarnings("ignore")
#matplot lib notebooks display command
%matplotlib inline

#SET OPTIONS
pd.options.display.float_format = '{:,.2f}'.format
#GLOBAL VARIABLES
today = datetime.date.today()
#cwd = os.getcwd()

In [394]:
#load dataset
non_auto_df = pd.read_csv("../Data/non_automated_invoices_with_acct_info_sample.csv", encoding='utf-8')
non_auto_df.dtypes

INVOICE_ID            object
ACCT_ID               object
CR_CHK_CD             object
CUST_STAT             object
PYDX_SCRE_NUM        float64
INDUSTRY_NM           object
CURR_NUM_LICENSES    float64
TRUE_LOGIN           float64
USAGE_SCORE          float64
EMP_CNT              float64
CRNCY_ISO_CD          object
CUSTOMER_REVENUE     float64
COLLECTOR             object
COLLECTION_STATUS     object
HAS_TAX_EXEMP           bool
ESCALATION_STATUS     object
BILLING_COUNTRY       object
BILLING_LANGUAGE      object
PAYMENT_TERM          object
PAYMENT_TYPE          object
HAS_30DAY_EXTXN         bool
PAST_DUE_FLG            bool
LATE_PMT_GT_30          bool
DAYS_TO_PAY            int64
CURRENCY_CODE         object
TOTAL_AMOUNT         float64
dtype: object

# Feature cleaning

### CR_CHK_CD

In [395]:
for el in pd.unique(non_auto_df['CR_CHK_CD']):
    print(el)

Prepayment
Approved
0
Defer Revenue
Credit Data Not Found
Credit Hold
Payment Plan
Bankruptcy
Not Required
Write Off - Bad Debt


In [396]:
'''
Drop 0 values
'''

#non_auto_df = non_auto_df.drop(index = non_auto_df.loc[lambda df: df['CR_CHK_CD'] == '0'].index)
#non_auto_df = non_auto_df.drop(index = non_auto_df.loc[lambda df: df['CR_CHK_CD'] == 'Bankruptcy'].index)
#non_auto_df = non_auto_df.drop(index = non_auto_df.loc[lambda df: df['CR_CHK_CD'] == 'Write Off - Bad Debt'].index)

'\nDrop 0 values\n'

### CUST_STAT

In [397]:
for el in pd.unique(non_auto_df['CUST_STAT']):
    print(el)

Direct Customer
Attrited Customer
Indirect Customer
Prospect


### INDUSTRY_NM

In [398]:
for el in pd.unique(non_auto_df['INDUSTRY_NM']):
    print(el)


Agriculture & Mining
Healthcare & Life Sciences
Retail & CG
Financial Services
Public Sector
Manufacturing
Communications & Media
Engineering, Construction, & Real Estate
High Tech
Professional Services
Education
Other
Travel, Transportation, & Hospitality
0
High_Tech
Nonprofit Institutions
Media & Communications
ITTelecoms
Media
High-Tech Software/Services
Health_Care
Professional_Services
Computer Software
Fraud
Energy & Utilities
Computer hardware/software/technology
Consulting
Higher Education
Software & Programming
Communications
Healthcare
Public_Sector
Technology
Computer Services
Media & Telecom
Technology - Software & Services
Payment
Automotive
Public Sector and Government
Consumer Products & Retail
Business Services
Financial_Services
Retail & Consumer Goods
Software & Technology
Hospitality, Travel, & Transportation
Ratings & Reviews
Aerospace & Defense
IT／通信
Communications Services
Retail_Consumer


In [399]:
'''
Drop 0 values
'''

#non_auto_df = non_auto_df.drop(index = non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == '0'].index)
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == '0']

,INVOICE_ID,ACCT_ID,CR_CHK_CD,CUST_STAT,PYDX_SCRE_NUM,INDUSTRY_NM,CURR_NUM_LICENSES,TRUE_LOGIN,USAGE_SCORE,EMP_CNT,...,BILLING_COUNTRY,BILLING_LANGUAGE,PAYMENT_TERM,PAYMENT_TYPE,HAS_30DAY_EXTXN,PAST_DUE_FLG,LATE_PMT_GT_30,DAYS_TO_PAY,CURRENCY_CODE,TOTAL_AMOUNT
170,a2d0M000002GtBMQA0,0013000000hIboVAAS,0,Prospect,0.00,0,0.00,0.00,0.00,3.00,...,US,English,Net30,Check,True,False,True,39,USD,275.76
443,a2d0M000002xMdrQAE,0013000001CAw5bAAD,Prepayment,Prospect,10.00,0,0.00,0.00,0.00,75.00,...,CN,English,Net30,Wire Transfer,True,False,True,102,USD,"8,617.50"
1057,a2d0M000002LqODQA0,0010M00001SmzmnQAB,0,Direct Customer,0.00,0,7.00,56.00,56.00,30.00,...,AE,English,Net30,Wire Transfer,True,False,True,35,USD,"1,117.31"
1437,a2d0M00000385CpQAI,0013000001FiulzAAB,Prepayment,Prospect,0.00,0,0.00,0.00,0.00,150.00,...,NL,English,Net30,Wire Transfer,True,False,False,-16,EUR,"4,137.14"
1512,a2d0M000002CXqKQAW,0010M00001U1YAnQAN,0,Direct Customer,0.00,0,5.00,25.00,58.00,13.00,...,SA,English,Net30,Wire Transfer,True,False,False,18,USD,"2,757.60"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97041,a2d0M000001neUvQAI,0010M00001SmqvtQAB,0,Direct Customer,0.00,0,19.00,73.00,89.00,3.00,...,FR,French,Net30,Wire Transfer,True,False,False,-28,EUR,461.83
97090,a2d0M000001ov1DQAQ,0010M00001UMEsQQAX,0,Prospect,0.00,0,0.00,0.00,0.00,101.00,...,ES,English,Net30,Check,True,False,True,61,USD,766.00
97196,a2d0M000001gc9cQAA,0010M00001QkpayQAB,0,Attrited Customer,0.00,0,0.00,0.00,58.00,5.00,...,JP,Japanese,Net30EOM,Wire Transfer,True,False,False,0,JPY,"1,132.23"
97232,a2d0M000002xwptQAA,0010M00001WYSiPQAX,0,Direct Customer,0.00,0,0.00,0.00,84.00,55.00,...,US,English,Net30,Check,True,False,True,36,USD,"3,592.54"


In [400]:
'''
“Communications & Media” ==== “Media & Communications” === “Media” ====“Media & Telecom”===“Communications Services”====“ITTelecoms” = “Media & Telecom” = “IT／通信”
'''
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'IT／通信']='Media_and_Telecom'
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Media & Telecom']='Media_and_Telecom'
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Communications & Media']='Media_and_Telecom'
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Media & Communications']='Media_and_Telecom'
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Media']='Media_and_Telecom'
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Media & Telecom']='Media_and_Telecom'
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Communications Services']='Media_and_Telecom'
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'ITTelecoms']='Media_and_Telecom'

In [401]:
'''
Healthcare and Life Sciences = “Health_Care” = Healthcare & Life Sciences = Healthcare
'''
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Healthcare and Life Sciences']='Health_Care'
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Healthcare & Life Sciences']='Health_Care'
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Healthcare']='Health_Care'


In [402]:
'''
Retail and CG = Consumer Products & Retail = Retail & Consumer Goods = “Retail_Consumer” == Retail & CG
'''
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Retail and CG']='Retail_Consumer'
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Consumer Products & Retail']='Retail_Consumer'
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Retail & Consumer Goods']='Retail_Consumer'
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Retail & CG']='Retail_Consumer'



In [403]:
'''
Financial Services = Financial_Services
'''
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Financial Services']='Financial_Services'


In [404]:
'''
Public sector == “Public Sector & Government” === Public Sector and Government ==== Public_Sector == Public Sector
'''
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Public Sector & Government']='Public sector'
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Public Sector and Government']='Public sector'
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Public_Sector']='Public sector'
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Public Sector']='Public sector'

In [405]:
'''
“High_Tech” === “High-Tech Software/Services” == “Computer Software”. === 
“Computer hardware/software/technology” == “Software & Programming” === “Technology - Software & Services”
==== Software & Technology === High Tech === Technology === Computer Services
'''
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'High-Tech Software/Services']='High_Tech'
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Computer Software']='High_Tech'
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Computer hardware/software/technology']='High_Tech'
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Software & Programming']='High_Tech'
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Technology - Software & Services']='High_Tech'
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Software & Technology']='High_Tech'
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'High Tech']='High_Tech'
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Technology']='High_Tech'
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Computer Services']='High_Tech'

In [406]:
'''
“Professional Services” = “Professional_Services”
'''
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Professional Services']='Professional_Services'


In [407]:
'''
Education ==== Higher Education
'''
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Higher Education']='Education'


In [408]:
'''
Travel, Transportation, & Hospitality======Hospitality, Travel, & Transportation
'''
non_auto_df.loc[lambda df: df['INDUSTRY_NM'] == 'Travel, Transportation, & Hospitality']='Hospitality, Travel, & Transportation'

In [409]:
for el in pd.unique(non_auto_df['INDUSTRY_NM']):
    print(el)


Agriculture & Mining
Health_Care
Retail_Consumer
Financial_Services
Public sector
Manufacturing
Media_and_Telecom
Engineering, Construction, & Real Estate
High_Tech
Professional_Services
Education
Other
Hospitality, Travel, & Transportation
0
Nonprofit Institutions
Fraud
Energy & Utilities
Consulting
Communications
Payment
Automotive
Business Services
Ratings & Reviews
Aerospace & Defense


### CRNCY_ISO_CD

In [410]:

for el in pd.unique(non_auto_df['CRNCY_ISO_CD']):
    print(el)


USD
Health_Care
Retail_Consumer
Financial_Services
Public sector
Media_and_Telecom
High_Tech
Professional_Services
EUR
JPY
Hospitality, Travel, & Transportation
AUD
CAD
GBP
SEK
Education


In [411]:
non_auto_df.loc[lambda df: df['CRNCY_ISO_CD'] == 'Health_Care']

,INVOICE_ID,ACCT_ID,CR_CHK_CD,CUST_STAT,PYDX_SCRE_NUM,INDUSTRY_NM,CURR_NUM_LICENSES,TRUE_LOGIN,USAGE_SCORE,EMP_CNT,...,BILLING_COUNTRY,BILLING_LANGUAGE,PAYMENT_TERM,PAYMENT_TYPE,HAS_30DAY_EXTXN,PAST_DUE_FLG,LATE_PMT_GT_30,DAYS_TO_PAY,CURRENCY_CODE,TOTAL_AMOUNT
1,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,...,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care
7,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,...,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care
24,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,...,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care
27,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,...,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care
44,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,...,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97263,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,...,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care
97285,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,...,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care
97290,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,...,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care
97317,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,...,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care,Health_Care


In [412]:
non_auto_df = non_auto_df.drop(index = non_auto_df.loc[lambda df: df['CRNCY_ISO_CD'] == 'Health_Care'].index)

In [413]:
non_auto_df.loc[lambda df: df['CRNCY_ISO_CD'] == 'Health_Care']

,INVOICE_ID,ACCT_ID,CR_CHK_CD,CUST_STAT,PYDX_SCRE_NUM,INDUSTRY_NM,CURR_NUM_LICENSES,TRUE_LOGIN,USAGE_SCORE,EMP_CNT,...,BILLING_COUNTRY,BILLING_LANGUAGE,PAYMENT_TERM,PAYMENT_TYPE,HAS_30DAY_EXTXN,PAST_DUE_FLG,LATE_PMT_GT_30,DAYS_TO_PAY,CURRENCY_CODE,TOTAL_AMOUNT


In [414]:
non_auto_df = non_auto_df.drop(index = non_auto_df.loc[lambda df: df['CRNCY_ISO_CD'] == 'Retail_Consumer'].index)

In [415]:
non_auto_df = non_auto_df.drop(index = non_auto_df.loc[lambda df: df['CRNCY_ISO_CD'] == 'Financial_Services'].index)

In [416]:
non_auto_df = non_auto_df.drop(index = non_auto_df.loc[lambda df: df['CRNCY_ISO_CD'] == 'Public sector'].index)

In [417]:
non_auto_df = non_auto_df.drop(index = non_auto_df.loc[lambda df: df['CRNCY_ISO_CD'] == 'Media_and_Telecom'].index)

In [418]:
non_auto_df = non_auto_df.drop(index = non_auto_df.loc[lambda df: df['CRNCY_ISO_CD'] == 'High_Tech'].index)

In [419]:
non_auto_df = non_auto_df.drop(index = non_auto_df.loc[lambda df: df['CRNCY_ISO_CD'] == 'Professional_Services'].index)

In [420]:
non_auto_df = non_auto_df.drop(index = non_auto_df.loc[lambda df: df['CRNCY_ISO_CD'] == 'Hospitality, Travel, & Transportation'].index)

In [421]:
non_auto_df = non_auto_df.drop(index = non_auto_df.loc[lambda df: df['CRNCY_ISO_CD'] == 'Education'].index)

In [422]:

for el in pd.unique(non_auto_df['CRNCY_ISO_CD']):
    print(el)


USD
EUR
JPY
AUD
CAD
GBP
SEK


### COLLECTION_STATUS

In [423]:

for el in pd.unique(non_auto_df['COLLECTION_STATUS']):
    print(el)


Non Responsive
Promise To Pay
NONE
Courtesy Contact
Willingness To Pay
Refusal To Pay
Invoice Correction Required
Non-Responsive
E-invoicing
Tax
Financial Inability To Pay
Red Account


In [424]:
'''
Non Responsive ==== Non-Responsive
'''
non_auto_df.loc[lambda df: df['COLLECTION_STATUS'] == 'Non Responsive']='Non-Responsive'


In [425]:
non_auto_df.loc[lambda df: df['COLLECTION_STATUS'] == 'Tax']

,INVOICE_ID,ACCT_ID,CR_CHK_CD,CUST_STAT,PYDX_SCRE_NUM,INDUSTRY_NM,CURR_NUM_LICENSES,TRUE_LOGIN,USAGE_SCORE,EMP_CNT,...,BILLING_COUNTRY,BILLING_LANGUAGE,PAYMENT_TERM,PAYMENT_TYPE,HAS_30DAY_EXTXN,PAST_DUE_FLG,LATE_PMT_GT_30,DAYS_TO_PAY,CURRENCY_CODE,TOTAL_AMOUNT
1089,a2d0M0000037Rm4QAE,0013000001Jz8LdAAJ,0,Direct Customer,0.00,Manufacturing,53.00,89.00,86.00,"2,200.00",...,BR,Portuguese (Brazilian),Net30,Check,True,False,True,33,USD,284.60
2907,a2d0M0000038ZRQQA2,0010M00001QqVxcQAF,0,Direct Customer,0.00,Manufacturing,79.00,68.00,83.00,"1,000.00",...,BR,Portuguese (Brazilian),Net30,Wire Transfer,True,False,True,39,USD,"4,236.59"
4335,a2d0M000001xeqfQAA,0013000000rpFcOAAU,Approved,Direct Customer,200.00,Manufacturing,39.00,82.00,82.00,50.00,...,CO,English,Net30,Wire Transfer,True,False,True,168,USD,"2,758.75"
4648,a2d0M000002yVJaQAM,0010M00001YFi4LQAT,0,Direct Customer,0.00,Manufacturing,27.00,49.00,92.00,"1,000.00",...,GT,English,Net30,Wire Transfer,True,False,True,31,USD,"3,527.12"
5266,a2d0M0000039m1rQAA,0010M00001Sq2szQAB,0,Direct Customer,0.00,Manufacturing,5.00,57.00,85.00,35.00,...,BR,English,Net30,Wire Transfer,True,False,False,10,USD,"1,499.44"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93590,a2d0M000002GlxqQAC,00130000016jnMfAAI,0,Direct Customer,0.00,Manufacturing,8.00,62.00,84.00,7.00,...,BR,Portuguese (Brazilian),Net30,Wire Transfer,True,False,True,33,USD,"1,628.71"
93769,a2d0M0000037PqhQAE,0010M00001QqxENQAZ,0,Direct Customer,0.00,"Engineering, Construction, & Real Estate",40.00,82.00,86.00,290.00,...,BR,Portuguese (Brazilian),Net30,Wire Transfer,True,False,True,40,USD,669.29
95026,a2d0M000002alwsQAA,0010M00001QkVqPQAV,0,Direct Customer,0.00,Agriculture & Mining,30.00,53.00,61.00,500.00,...,AR,Spanish,Net30,Wire Transfer,True,False,True,72,USD,"20,682.00"
95385,a2d0M000002warhQAA,0010M00001QqcO9QAJ,0,Direct Customer,0.00,Manufacturing,33.00,50.00,56.00,500.00,...,BR,Portuguese (Brazilian),Net30,Wire Transfer,True,False,False,13,USD,"4,828.73"


### ESCALATION_STATUS

In [426]:


for el in pd.unique(non_auto_df['ESCALATION_STATUS']):
    print(el)


Non-Responsive
NONE
Pending AE Response/Involvement
Awaiting AE Involvement/Action
Pending Management Action
Assigned To Bad Debt
Escalated To Order Management


In [427]:
non_auto_df.loc[lambda df: df['ESCALATION_STATUS'] == 'Assigned To Bad Debt']

,INVOICE_ID,ACCT_ID,CR_CHK_CD,CUST_STAT,PYDX_SCRE_NUM,INDUSTRY_NM,CURR_NUM_LICENSES,TRUE_LOGIN,USAGE_SCORE,EMP_CNT,...,BILLING_COUNTRY,BILLING_LANGUAGE,PAYMENT_TERM,PAYMENT_TYPE,HAS_30DAY_EXTXN,PAST_DUE_FLG,LATE_PMT_GT_30,DAYS_TO_PAY,CURRENCY_CODE,TOTAL_AMOUNT
54801,a2d0M000002Go1IQAS,0010M00001UEg5aQAD,Prepayment,Prospect,10.00,Manufacturing,0.00,0.00,86.00,50.00,...,US,English,Net30,Check,True,False,True,126,USD,"13,325.99"


In [428]:
non_auto_df = non_auto_df.drop(index = non_auto_df.loc[lambda df: df['ESCALATION_STATUS'] == 'Assigned To Bad Debt'].index)

### BILLING_COUNTRY

In [429]:

for el in pd.unique(non_auto_df['BILLING_COUNTRY']):
    print(el)

Non-Responsive
US
GB
JP
AU
BR
ES
IT
IL
IN
TH
CA
TW
BE
FI
NO
AE
DE
NL
CN
HK
CO
FR
DK
PT
AT
CH
CL
SG
SA
SE
SI
MX
PH
ID
LU
IE
RU
SN
KE
PE
EG
CZ
ZA
PL
NZ
KR
PA
RO
GT
BG
VN
LB
HU
DO
MK
AR
PR
CR
HR
MT
MY
LI
GR
MA
CY
BY
QA
PK
NG
LT
MU
SV
EC
TT
ZM
LK
TR
BH
JE
KH
AD
JO
SK
TZ
EE
BA
IS
MC
VE
GE
UA
LS
PY
CM
OM
TN
RS
NI
UG
GH


### BILLING_LANGUAGE 

In [430]:

for el in pd.unique(non_auto_df['BILLING_LANGUAGE']):
    print(el)


Non-Responsive
English
Japanese
Portuguese (Brazilian)
Spanish
Italian
French
German
Chinese (Simplified)
Korean
Dutch
Swedish


### PAYMENT_TERM 

In [431]:

for el in pd.unique(non_auto_df['PAYMENT_TERM']):
    print(el)

Non-Responsive
Net30
Net60
Net30EOM
Net45
Net0
Net60EOM


### PAYMENT_TYPE

In [432]:

for el in pd.unique(non_auto_df['PAYMENT_TYPE']):
    print(el)

Non-Responsive
Check
Wire Transfer


### CURRENCY_CODE

In [433]:
'''
IS this redundant with CurrencyISO?
'''
for el in pd.unique(non_auto_df['CURRENCY_CODE']):
    print(el)

Non-Responsive
USD
GBP
JPY
AUD
EUR
CAD
SEK


In [434]:
### DAYS_TO_PAY

In [435]:
for el in pd.unique(non_auto_df['DAYS_TO_PAY']):
    print(el)

Non-Responsive
95
-59
5
10
13
-9
2
8
36
51
79
-17
-20
3
11
40
25
123
-12
41
15
12
19
6
29
-1
-4
-2
27
46
-3
16
1
-25
-10
84
61
17
38
53
9
55
20
117
32
22
60
70
42
4
82
37
102
67
26
34
113
33
43
49
-27
65
-23
21
-8
45
39
7
54
56
-6
528
72
14
44
-5
210
88
62
35
63
179
52
24
68
251
112
57
23
50
-7
-11
48
28
0
-16
-13
121
81
18
74
-24
47
-19
89
160
31
246
490
77
119
80
203
59
-35
-28
143
92
122
91
58
66
73
93
71
30
87
86
120
-15
180
149
147
130
125
-18
111
99
168
148
105
126
153
-14
76
140
375
78
502
-22
190
-30
100
186
-21
98
103
144
109
97
135
193
136
292
127
101
173
-26
-44
138
315
145
199
132
108
141
129
96
85
-33
202
75
-29
169
-45
277
114
154
128
69
-51
221
280
137
253
281
182
234
390
116
-42
386
268
146
155
319
64
164
191
163
-43
90
106
240
497
183
83
167
350
216
104
282
171
-34
187
241
214
487
94
184
142
157
269
-37
124
327
493
176
-31
222
170
185
332
110
195
166
192
257
174
156
133
152
118
392
219
115
-40
198
151
224
495
485
150
178
-32
107
261
340
177
188
209
242
134
320
-36
211


In [436]:
# CUTOFF AT 270 days (9 months) and non-responsive 
non_auto_df = non_auto_df.drop(index = non_auto_df.loc[lambda df: df['DAYS_TO_PAY'] == 'Non-Responsive'].index)
non_auto_df = non_auto_df.drop(index = non_auto_df.loc[lambda df: df['DAYS_TO_PAY'] >270].index)

In [437]:
for el in pd.unique(non_auto_df['DAYS_TO_PAY']):
    print(el)

95
-59
5
10
13
-9
2
8
36
51
79
-17
-20
3
11
40
25
123
-12
41
15
12
19
6
29
-1
-4
-2
27
46
-3
16
1
-25
-10
84
61
17
38
53
9
55
20
117
32
22
60
70
42
4
82
37
102
67
26
34
113
33
43
49
-27
65
-23
21
-8
45
39
7
54
56
-6
72
14
44
-5
210
88
62
35
63
179
52
24
68
251
112
57
23
50
-7
-11
48
28
0
-16
-13
121
81
18
74
-24
47
-19
89
160
31
246
77
119
80
203
59
-35
-28
143
92
122
91
58
66
73
93
71
30
87
86
120
-15
180
149
147
130
125
-18
111
99
168
148
105
126
153
-14
76
140
78
-22
190
-30
100
186
-21
98
103
144
109
97
135
193
136
127
101
173
-26
-44
138
145
199
132
108
141
129
96
85
-33
202
75
-29
169
-45
114
154
128
69
-51
221
137
253
182
234
116
-42
268
146
155
64
164
191
163
-43
90
106
240
183
83
167
216
104
171
-34
187
241
214
94
184
142
157
269
-37
124
176
-31
222
170
185
110
195
166
192
257
174
156
133
152
118
219
115
-40
198
151
224
150
178
-32
107
261
177
188
209
242
134
-36
211
196
258
-49
161
238
201
259
226
247
228
232
208
-39
197
172
131
218
205
243
213
162
236
-53
139
-41
270
-46
158

## Change back to correct dtypes

In [438]:
non_auto_df.dtypes

INVOICE_ID           object
ACCT_ID              object
CR_CHK_CD            object
CUST_STAT            object
PYDX_SCRE_NUM        object
INDUSTRY_NM          object
CURR_NUM_LICENSES    object
TRUE_LOGIN           object
USAGE_SCORE          object
EMP_CNT              object
CRNCY_ISO_CD         object
CUSTOMER_REVENUE     object
COLLECTOR            object
COLLECTION_STATUS    object
HAS_TAX_EXEMP        object
ESCALATION_STATUS    object
BILLING_COUNTRY      object
BILLING_LANGUAGE     object
PAYMENT_TERM         object
PAYMENT_TYPE         object
HAS_30DAY_EXTXN      object
PAST_DUE_FLG         object
LATE_PMT_GT_30       object
DAYS_TO_PAY          object
CURRENCY_CODE        object
TOTAL_AMOUNT         object
dtype: object

Should be: 
    
INVOICE_ID            object
ACCT_ID               object
CR_CHK_CD             object
CUST_STAT             object
PYDX_SCRE_NUM        float64
INDUSTRY_NM           object
CURR_NUM_LICENSES    float64
TRUE_LOGIN           float64
USAGE_SCORE          float64
EMP_CNT              float64
CRNCY_ISO_CD          object
CUSTOMER_REVENUE     float64
COLLECTOR             object
COLLECTION_STATUS     object
HAS_TAX_EXEMP           bool
ESCALATION_STATUS     object
BILLING_COUNTRY       object
BILLING_LANGUAGE      object
PAYMENT_TERM          object
PAYMENT_TYPE          object
HAS_30DAY_EXTXN         bool
PAST_DUE_FLG            bool
LATE_PMT_GT_30          bool
DAYS_TO_PAY            int64
CURRENCY_CODE         object
TOTAL_AMOUNT         float64
dtype: object

In [439]:
'''
Recast after modifying
'''
non_auto_df.loc[:,'PYDX_SCRE_NUM'] = non_auto_df.loc[:,'PYDX_SCRE_NUM'].astype('float64')
non_auto_df[['PYDX_SCRE_NUM', 'CURR_NUM_LICENSES', 'TRUE_LOGIN', 'USAGE_SCORE',\
             'EMP_CNT', 'CUSTOMER_REVENUE','DAYS_TO_PAY', 'TOTAL_AMOUNT' \
            ]] = non_auto_df[['PYDX_SCRE_NUM', 'CURR_NUM_LICENSES', 'TRUE_LOGIN', \
                              'USAGE_SCORE', 'EMP_CNT', 'CUSTOMER_REVENUE','DAYS_TO_PAY', 'TOTAL_AMOUNT'\
            ]].astype('float64')

non_auto_df[['HAS_TAX_EXEMP', 'HAS_30DAY_EXTXN', 'PAST_DUE_FLG', 'LATE_PMT_GT_30']] = \
non_auto_df[['HAS_TAX_EXEMP', 'HAS_30DAY_EXTXN', 'PAST_DUE_FLG', 'LATE_PMT_GT_30']].astype('bool')
non_auto_df.dtypes

INVOICE_ID            object
ACCT_ID               object
CR_CHK_CD             object
CUST_STAT             object
PYDX_SCRE_NUM        float64
INDUSTRY_NM           object
CURR_NUM_LICENSES    float64
TRUE_LOGIN           float64
USAGE_SCORE          float64
EMP_CNT              float64
CRNCY_ISO_CD          object
CUSTOMER_REVENUE     float64
COLLECTOR             object
COLLECTION_STATUS     object
HAS_TAX_EXEMP           bool
ESCALATION_STATUS     object
BILLING_COUNTRY       object
BILLING_LANGUAGE      object
PAYMENT_TERM          object
PAYMENT_TYPE          object
HAS_30DAY_EXTXN         bool
PAST_DUE_FLG            bool
LATE_PMT_GT_30          bool
DAYS_TO_PAY          float64
CURRENCY_CODE         object
TOTAL_AMOUNT         float64
dtype: object

In [440]:
#Normalize Numeric fields

def pre_process(df, dropcols=None):
    '''
    The pre_process function that will return a transformed dataframe
    Notes:
        1) dropcols should be a list
        2) integer varibles are transformed using mean normalization
    '''
    #drop columns input by the user
    if dropcols is not None:
        df = df.drop(columns=dropcols)
    print("columns dropped")
    
    #loop through remaining columns and apply the appropreate transform
    for i in df.columns:
        print(i)
        '''
        #mean normalization for int variables
        if df[i].dtype == "int64" or df[i].dtype == 'float64':
            mean = df[i].mean()
            if (df[i].max() - df[i].min()) == 0:
                denom = 1
                print("useless int column", i)
                break
            else:
                denom = (df[i].max() - df[i].min())
            df[i] = df[i].apply(lambda x: float(x - mean) / denom)
        '''
        #convert bool values to 1 and 0
        if df[i].dtype == "bool":
            df[i] = df[i].map({True: 1, False: 0})
        elif df[i].dtype == "object":
            df = pd.get_dummies(df, columns=[i], prefix = [i], dtype="int8") #, sparse=True
        
    return df



In [441]:
print("executed")
clean_df = pre_process(non_auto_df, dropcols=['INVOICE_ID', 'ACCT_ID', 'CURRENCY_CODE', 'LATE_PMT_GT_30'])
#clean_df.to_csv('../Data/clean_df.csv')
#dtypes_df = clean_df.dtypes.to_frame('dtypes').reset_index()
#dtypes_df.to_csv('../Data/clean_dtypes.csv')
print("done")
clean_df = clean_df.drop(columns = 'Unnamed: 0')

executed
columns dropped
CR_CHK_CD
CUST_STAT
PYDX_SCRE_NUM
INDUSTRY_NM
CURR_NUM_LICENSES
TRUE_LOGIN
USAGE_SCORE
EMP_CNT
CRNCY_ISO_CD
CUSTOMER_REVENUE
COLLECTOR
COLLECTION_STATUS
HAS_TAX_EXEMP
ESCALATION_STATUS
BILLING_COUNTRY
BILLING_LANGUAGE
PAYMENT_TERM
PAYMENT_TYPE
HAS_30DAY_EXTXN
PAST_DUE_FLG
DAYS_TO_PAY
TOTAL_AMOUNT
done


In [442]:
clean_df.to_csv('../Data/clean_with_collector_df.csv')

print("done")

SystemError: <built-in method item of numpy.ndarray object at 0x131190a30> returned a result with an error set